In [1]:
import numpy as np
import pandas as pd

import gc

from source.process_ts import melt_merge, lag_feats, rolling_sum, reduce_mem_usage, agg_id

pd.set_option('max_columns', 100)

In [2]:
sales = pd.read_csv('data/raw/sales_train_validation.csv')
calendar = pd.read_csv('data/raw/calendar.csv')
prices = pd.read_csv('data/raw/sell_prices.csv')

sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,5,3,1,0,0,0,1,2,3,0,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,0,3,1,1,2,1,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [3]:
result = melt_merge(sales, calendar, prices)

result.head()

Mem. usage decreased to  0.05 Mb (70.5% reduction)
Mem. usage decreased to 3060.88 Mb (24.6% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,week,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN


In [4]:
result.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 21 columns):
 #   Column        Non-Null Count     Dtype         
---  ------        --------------     -----         
 0   id            58327370 non-null  category      
 1   item_id       58327370 non-null  object        
 2   dept_id       58327370 non-null  category      
 3   cat_id        58327370 non-null  category      
 4   store_id      58327370 non-null  object        
 5   state_id      58327370 non-null  category      
 6   d             58327370 non-null  int16         
 7   sales         58327370 non-null  int16         
 8   date          58327370 non-null  datetime64[ns]
 9   wday          58327370 non-null  int8          
 10  month         58327370 non-null  int8          
 11  year          58327370 non-null  int16         
 12  event_name_1  4695460 non-null   category      
 13  event_type_1  4695460 non-null   category      
 14  event_name_2  121960 non-null   

In [5]:
del sales
del calendar
del prices

gc.collect()

0

In [6]:
result.to_pickle('data/processed/melted.pkl')

# Lags features

In [3]:
result = pd.read_pickle('data/processed/melted.pkl')
result = result[['id', 'd', 'sales']].copy()
gc.collect()

0

In [4]:
np.arange(7, 14, 1)

array([ 7,  8,  9, 10, 11, 12, 13])

In [5]:
result = lag_feats(result, lags=np.arange(7, 14, 1))

result.head()

Mem. usage decreased to 1558.99 Mb (60.0% reduction)


,id,d,sales,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13
0,HOBBIES_1_001_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20334,FOODS_2_214_TX_3_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20333,FOODS_2_213_TX_3_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20332,FOODS_2_212_TX_3_validation,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20331,FOODS_2_211_TX_3_validation,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
result.to_pickle('data/processed/lags7_13.pkl')

In [7]:
result = result[['id', 'd', 'sales']].copy()

gc.collect()

result = lag_feats(result, lags=np.arange(14, 21, 1))

result.to_pickle('data/processed/lags14_20.pkl')

result.head()

Mem. usage decreased to 1558.99 Mb (60.0% reduction)


,id,d,sales,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20
0,HOBBIES_1_001_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,HOBBIES_1_014_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15244,FOODS_3_827_TX_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
result = result[['id', 'd', 'sales']].copy()

gc.collect()

result = lag_feats(result, lags=np.arange(21, 28, 1))

result.to_pickle('data/processed/lags21_27.pkl')

result.head()

Mem. usage decreased to 1558.99 Mb (60.0% reduction)


,id,d,sales,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27
0,HOBBIES_1_001_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20322,FOODS_2_202_TX_3_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15245,HOBBIES_1_001_TX_2_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20333,FOODS_2_213_TX_3_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20332,FOODS_2_212_TX_3_validation,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
result = result[['id', 'd', 'sales']].copy()

gc.collect()

result = lag_feats(result, lags=np.arange(28, 35, 1))

result.to_pickle('data/processed/lags28_34.pkl')

result.head()

Mem. usage decreased to 1558.99 Mb (60.0% reduction)


,id,d,sales,lag_28,lag_29,lag_30,lag_31,lag_32,lag_33,lag_34
0,HOBBIES_1_001_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,HOBBIES_1_013_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22866,HOUSEHOLD_2_428_WI_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15244,FOODS_3_827_TX_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Rolling stats

In [2]:
results = pd.read_pickle('data/processed/melted.pkl')

results.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,week,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN


In [3]:
results = results[['id', 'date', 'sales']]

gc.collect()

for period in ['7d', '14d', '21d', '28d']:
    tmp = rolling_sum(results, window=period)
    tmp.to_pickle(f'data/processed/rolling_{period}.pkl')
    del tmp
    gc.collect()

Mem. usage decreased to 668.99 Mb (33.3% reduction)
Mem. usage decreased to 668.99 Mb (33.3% reduction)
Mem. usage decreased to 668.99 Mb (33.3% reduction)
Mem. usage decreased to 668.99 Mb (33.3% reduction)


# Same period, a year before

In [2]:
results = pd.read_pickle('data/processed/melted.pkl')

results.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,week,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5,NaN


In [3]:
results = results[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 'month', 'year', 'week', 'sales']]

results.head()

,id,item_id,dept_id,cat_id,store_id,state_id,wday,month,year,week,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2011,5,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2011,5,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2011,5,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2011,5,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2011,5,0


In [4]:
to_merge = results[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'month', 'year', 'week']].drop_duplicates()

to_merge.head()

,id,item_id,dept_id,cat_id,store_id,state_id,month,year,week
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,2011,5
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,2011,5
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,2011,5
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,2011,5
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,2011,5


In [5]:
agg_dict = {'store_id': 'store', 
            'item_id': 'item', 
            'dept_id': 'dept', 
            'cat_id': 'cat', 
            'id': 'id'}

for key in agg_dict.keys():
    ly, ly_m, ly_w, lm, lw = agg_id(results, agg_dict[key], key)
    tmp_merge = to_merge[[key] + ['month', 'year', 'week']].drop_duplicates()
    df = pd.merge(tmp_merge, ly, on=[key, 'year'], how='left')
    df = pd.merge(df, ly_m, on=[key, 'year', 'month'], how='left')
    df = pd.merge(df, ly_w, on=[key, 'year', 'week'], how='left')
    df = pd.merge(df, lm, on=[key, 'year', 'month'], how='left')
    df = pd.merge(df, lw, on=[key, 'year', 'week'], how='left')
    df = reduce_mem_usage(df)
    df.to_pickle(f'data/processed/aggregate_{agg_dict[key]}.pkl')
    del df, tmp_merge
    gc.collect()

Mem. usage decreased to  0.09 Mb (50.0% reduction)
Mem. usage decreased to 28.87 Mb (50.0% reduction)
Mem. usage decreased to  0.05 Mb (56.4% reduction)
Mem. usage decreased to  0.02 Mb (56.5% reduction)
Mem. usage decreased to 232.47 Mb (55.4% reduction)


In [6]:
tmp = pd.read_pickle(f'data/processed/aggregate_store.pkl')

tmp.tail(30)

,store_id,month,year,week,store_last_year,store_monht_last_year,store_week_last_year,store_last_month,store_last_week
3280,CA_1,4,2016,15,1.444336,1.467773,1.477539,1.444336,1.573242
3281,CA_2,4,2016,15,1.085938,0.781250,0.803223,1.335938,1.498047
3282,CA_3,4,2016,15,2.044922,2.029297,2.199219,2.078125,2.154297
3283,CA_4,4,2016,15,0.786133,0.779785,0.785645,0.843262,0.880371
3284,TX_1,4,2016,15,1.034180,1.027344,1.092773,1.066406,1.130859
3285,TX_2,4,2016,15,1.226562,1.226562,1.275391,1.329102,1.329102
3286,TX_3,4,2016,15,1.207031,1.232422,1.259766,1.240234,1.279297
3287,WI_1,4,2016,15,1.145508,1.100586,1.081055,1.209961,1.266602
3288,WI_2,4,2016,15,1.354492,1.285156,1.378906,1.684570,2.009766
3289,WI_3,4,2016,15,1.071289,1.024414,1.090820,1.264648,1.387695


In [7]:
tmp.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3310 entries, 0 to 3309
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   store_id               3310 non-null   object 
 1   month                  3310 non-null   int8   
 2   year                   3310 non-null   int16  
 3   week                   3310 non-null   int8   
 4   store_last_year        2720 non-null   float16
 5   store_monht_last_year  2720 non-null   float16
 6   store_week_last_year   2670 non-null   float16
 7   store_last_month       3300 non-null   float16
 8   store_last_week        3290 non-null   float16
dtypes: float16(5), int16(1), int8(2), object(1)
memory usage: 97.0+ KB


# Putting all together

In [2]:
base = pd.read_pickle('data/processed/melted.pkl')

base.drop(['event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2'], axis=1, inplace=True)

base = reduce_mem_usage(base)

gc.collect()

base.head()

Mem. usage decreased to 2838.37 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,snap_CA,snap_TX,snap_WI,week,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN


In [3]:
base.shape

(58327370, 17)

In [5]:
for rol_p in ['7', '14', '21', '28']:
    rol = pd.read_pickle(f'data/processed/rolling_{rol_p}d.pkl')
    base = pd.merge(base, rol, on=['id', 'date'], how='left')
    del rol
    base = reduce_mem_usage(base)
    gc.collect()
    
base.head()

Mem. usage decreased to 2949.62 Mb (0.0% reduction)
Mem. usage decreased to 3060.88 Mb (0.0% reduction)
Mem. usage decreased to 3172.13 Mb (0.0% reduction)
Mem. usage decreased to 3283.38 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,snap_CA,snap_TX,snap_WI,week,sell_price,rol_7d,rol_14d,rol_21d,rol_28d
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN


In [6]:
base.to_pickle('data/processed/base_rolling.pkl')

In [3]:
base = pd.read_pickle('data/processed/base_rolling.pkl')

base.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,snap_CA,snap_TX,snap_WI,week,sell_price,rol_7d,rol_14d,rol_21d,rol_28d
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN


In [4]:
for aggr_p in ['cat', 'dept', 'item', 'store']:
    aggr = pd.read_pickle(f'data/processed/aggregate_{aggr_p}.pkl')
    col_merge = ['year', 'month', 'week', f'{aggr_p}_id']
    base = pd.merge(base, aggr, on=col_merge, how='left')
    del aggr
    base = reduce_mem_usage(base)
    gc.collect()
    
aggr = pd.read_pickle(f'data/processed/aggregate_id.pkl')
col_merge = ['year', 'month', 'week', 'id']
base = pd.merge(base, aggr, on=col_merge, how='left')
del aggr
base = reduce_mem_usage(base)
gc.collect()

base.head()

Mem. usage decreased to 3839.63 Mb (0.0% reduction)
Mem. usage decreased to 4395.88 Mb (0.0% reduction)
Mem. usage decreased to 4952.14 Mb (0.0% reduction)
Mem. usage decreased to 5508.39 Mb (0.0% reduction)
Mem. usage decreased to 6064.64 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,month,year,snap_CA,snap_TX,snap_WI,week,sell_price,rol_7d,rol_14d,rol_21d,rol_28d,cat_last_year,cat_monht_last_year,cat_week_last_year,cat_last_month,cat_last_week,dept_last_year,dept_monht_last_year,dept_week_last_year,dept_last_month,dept_last_week,item_last_year,item_monht_last_year,item_week_last_year,item_last_month,item_last_week,store_last_year,store_monht_last_year,store_week_last_year,store_last_month,store_last_week,id_last_year,id_monht_last_year,id_week_last_year,id_last_month,id_last_week
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
base.to_pickle('data/processed/base_nolags.pkl')

In [2]:
base = pd.read_pickle(f'data/processed/base_nolags.pkl')

base.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date'], axis=1, inplace=True)

gc.collect()

base.head()

,id,d,sales,wday,month,year,snap_CA,snap_TX,snap_WI,week,sell_price,rol_7d,rol_14d,rol_21d,rol_28d,cat_last_year,cat_monht_last_year,cat_week_last_year,cat_last_month,cat_last_week,dept_last_year,dept_monht_last_year,dept_week_last_year,dept_last_month,dept_last_week,item_last_year,item_monht_last_year,item_week_last_year,item_last_month,item_last_week,store_last_year,store_monht_last_year,store_week_last_year,store_last_month,store_last_week,id_last_year,id_monht_last_year,id_week_last_year,id_last_month,id_last_week
0,HOBBIES_1_001_CA_1_validation,1,0,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1,0,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1,0,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,1,0,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,1,0,1,1,2011,0,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for lag in ['7_13', '14_20', '21_27', '28_34']:
    lag_data = pd.read_pickle(f'data/processed/lags{lag}.pkl')
    del lag_data['sales']
    base = pd.merge(base, lag_data, on=['id', 'd'], how='left')
    del lag_data
    base = reduce_mem_usage(base)
    gc.collect()

base.head()